# Automates the reading, masking, and stacking of Sentinel-2 Imagery

I use this code after I download S2 imagery from Copernicus. (https://scihub.copernicus.eu/dhus/#/home)

Stacks only the visible bands, red edge, NIR, and SWIR.  This can be edited to include additional bands, but you will need to change the code.  If you can't figure this out, contact me at wegmueller@wisc.edu

WINDOWS USERS: You will need to change the "/" to "\" where indicated. The direction of the slash matters or the code won't work for you.

IMPORTANT: I mask out everthing but vegetation because that's what I use. This can easily be edited, and I've annoted where to do so.

In [1]:
import os
from osgeo import gdal, gdal_array
import numpy as np
import numpy.ma as ma
import rasterio
from rasterio.enums import Resampling

In [7]:
### This is the only block you will need to edit ###

# Create a list of image directories for every image you want
# processed. This is the only thing you'll need to do.
# Set the directory to the 'IMG_DATA' file. 
# Sentinel-2 should all be formatted the same, 
# which lets this code be more or less universal

dir_list = ['.../S2A_MSIL2A_20200713T170901_N0214_R112_T15TVL_20200713T213310.SAFE/GRANULE/L2A_T15TVL_A026421_20200713T171937/IMG_DATA'
            '.../S2A_MSIL2A_20200730T165901_N0214_R069_T15TVL_20200730T212533.SAFE/GRANULE/L2A_T15TVL_A026664_20200730T170629/IMG_DATA'
           ]
    

In [8]:
def rescale (scale_factor, dataset, ref_dataset, output_fp):
    
    scale_factor = scale_factor

    with rasterio.open(dataset) as dataset:

        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=Resampling.nearest #https://gisgeography.com/raster-resampling/
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
        
    ref_img = rasterio.open(ref_dataset)

    with rasterio.Env():
        profile = ref_img.profile
        with rasterio.open(output_fp, 'w', **profile) as dst:
            dst.write(data)
            
def rescale_mask (scale_factor, dataset, ref_dataset, output_fp):  ###NO LONGER USED
    
    scale_factor = scale_factor

    with rasterio.open(dataset) as dataset:

        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=Resampling.nearest #https://gisgeography.com/raster-resampling/
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
        
    ref_img = rasterio.open(ref_dataset)

    with rasterio.Env():
        profile = ref_img.profile
        profile['dtype'] = rasterio.uint8
        with rasterio.open(output_fp, 'w', **profile) as dst:
            dst.write(data)
            
def get_image_name(image_directory):
    file = fp + '/R10m'              #WINDOWS USERS: change slash direction here.
    names = os.listdir(file)[0]
    img_name = names[:22]
    return img_name



In [9]:
# WINDOWS USERS:  Change the direction of the slash where needed in
#                 all the strings in this block.

for fp in dir_list:
    os.chdir(fp)

    # Resample Red Edge (8A) and SWIR (12) bands from 20m to 10m
    # Resample QI for mask

    #Set this correctly to automate the rest of this script
    
    img_identifier = get_image_name(fp)

    re = 'R20m/%s_B8A_20m.jp2' %img_identifier 
    swir = 'R20m/%s_B12_20m.jp2' %img_identifier 

    qi = 'R20m/%s_SCL_20m.jp2'%img_identifier 

    ref_img = 'R10m/%s_B02_10m.jp2' %img_identifier 
    
    
    #Mask 20m bands
    #Create mask
    mask_file = gdal.Open(qi)
    mask_data = mask_file.GetRasterBand(1)
    mask_array = gdal_array.BandReadAsArray(mask_data)

    # good_pixels = [4] #vegetation
    # bad_pixels = [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11]  #no data, saturated/defective pixel, shadows, 
                                                        # cloud shadows, not_vegetated
                                                # water, cloud med prob, cloud high prob, cirrus, snow/ice

    # Here I am masking everything but vegetation.  This line can be adjusted
    # to mask out different things; the key is just above this line.
    # The mask is inverted, which is why vegetation is selected. 
    # Keep that in mind as you make changes. 
    mask = (mask_array == 4)
    
    mask_vals = np.unique(mask, return_counts=True)
    percent_masked = (mask_vals[1][0])/(sum(mask_vals[1]))
    print('Percent of image masked: ' + str(percent_masked))
        
    qa_mask = np.invert(mask) #Inverstion is necessary to work with ma.masked_array

    # Mask bands with 20m resolution
    images_20m = [re, swir]

    arr = np.zeros((np.shape(qa_mask)[0], np.shape(qa_mask)[1]),  dtype=np.uint16)  # pre-allocate
    
    #Set up a list of masked images to be incorperated into the stack
    masked_20m_images = []
    
    # apply QA mask and remove all bad pixels
    for image in images_20m:
        org_image = gdal.Open(image)
        
        masked_fp = '%s_masked.tif' %image[:-4]
        masked_20m_images.append(masked_fp)
        
        band_data = org_image.GetRasterBand(1)
        band_array = gdal_array.BandReadAsArray(band_data)
        
        tmp = ma.masked_array(band_array, qa_mask)
        arr = ma.filled(tmp, 0)  # fill mask using nodata value.
      
        # Write data to a geotiff
        with rasterio.open(image) as src:
            kwds = src.profile
            kwds['nodata'] = 0
            kwds['driver'] = 'GTiff'
            kwds['width'] = arr.shape[1]
            kwds['height'] = arr.shape[0]
            kwds['count'] = 1

            with rasterio.open(masked_fp, 'w', **kwds) as dst:
                 dst.write(np.array(arr).astype(rasterio.uint16), 1)
                
    resampled_20m = []
    
    for masked_image in masked_20m_images:
        rescaled_fp = masked_image[:-4] + 'resampled.tif'
        resampled_20m.append(rescaled_fp)
        
        rescale((2), masked_image, ref_img, rescaled_fp)

            
    # Stack desired 10m image bands
    band_list = ('R10m/%s_B02_10m.jp2' %img_identifier,
                 'R10m/%s_B03_10m.jp2' %img_identifier,
                 'R10m/%s_B04_10m.jp2' %img_identifier,
                 'R10m/%s_B08_10m.jp2' %img_identifier)

    stacked_fp = 'R10m/%s_stack.jp2' %img_identifier
    
    # Read metadata of first file
    with rasterio.open(band_list[0]) as src0:
        meta = src0.meta

    # Update meta to reflect the number of layers
    meta.update(count = len(band_list))

    # Read each layer and write it to stack
    with rasterio.open(stacked_fp, 'w', **meta) as dst:
        for id, layer in enumerate(band_list, start=1):
            with rasterio.open(layer) as src1:
                dst.write_band(id, src1.read(1))
                
    #Create mask using data from masked and resampled 20m band
    mask_file = gdal.Open(resampled_20m[0])
    mask_data = mask_file.GetRasterBand(1)
    mask_array = gdal_array.BandReadAsArray(mask_data)

    mask = (mask_array == 0) #No data is zero
        
    bands = [1,2,3,4]

    arr = np.zeros((len(bands), np.shape(mask)[0], np.shape(mask)[1]), dtype=np.uint16)  # pre-allocate
    
    stack = gdal.Open(stacked_fp)
    masked_fp = 'R10m/%s_rgb_stack_masked.tif' %img_identifier

    # apply mask
    for count, b in enumerate(bands):
        band_data = stack.GetRasterBand(b)
        band_array = gdal_array.BandReadAsArray(band_data)

        tmp = ma.masked_array(band_array, mask)

        arr[count,:,:] = ma.filled(tmp, 0)  # fill mask using nodata value. add to 'arr' to create a 4-band image

    # Write data to a geotiff
    with rasterio.open(ref_img) as src:
        kwds = src.profile
        kwds['nodata'] = 0
        kwds['driver'] = 'GTiff'
        kwds['width'] = arr.shape[2]
        kwds['height'] = arr.shape[1]
        kwds['count'] = arr.shape[0]

        with rasterio.open(masked_fp, 'w', **kwds) as dst:
             dst.write(np.array(arr).astype(rasterio.uint16))
                
    #Stack visible bands with resampled and masked 20m bands
    new_band_count = len(bands) + len(resampled_20m)
    
    stack_writing = rasterio.open(masked_fp)
    
    profile = stack_writing.profile
    profile.update(count=new_band_count)
    
    stack_writing = None

    src1 = rasterio.open(masked_fp)
    n = 0
    
    full_stack = '%s_full_masked_stack.tif' %img_identifier

    with rasterio.open(full_stack, 'w', **profile) as dst:
        for band in range(1, new_band_count+1):
            if band <= len(bands):
                band_array = src1.read(band)
                band_array = band_array.astype(rasterio.uint16)
                dst.write(band_array, band)
            else:
                resampled_20m_band = rasterio.open(resampled_20m[n])
                band_array = resampled_20m_band.read(1)
                dst.write(band_array, band)
                n += 1

Percent of image masked: 0.40470157696888864
